# Test Load Daily

Start by loading one "new" tile.

## Imports

In [1]:
import os
import numpy as np
from astropy import __version__ as astropy_version
from astropy.table import Table, join
from sqlalchemy import __version__ as sqlalchemy_version
from sqlalchemy.exc import IntegrityError
from desiutil import __version__ as desiutil_version
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger, DEBUG
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desispec.io.meta import findfile
from desispec.scripts.zcatalog import read_redrock
from desispec.zcatalog import find_primary_spectra
from specprodDB import __version__ as specprodDB_version
import specprodDB.load as db
from specprodDB.util import no_sky
# from specprodDB.util import spgrpid, surveyid, no_sky

## Initial Values

In [2]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
# tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
overwrite = False

## Initialize Database

In [3]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    versions = [db.Version(package='specprod-db', version=specprodDB_version),
                db.Version(package='lsdr9-photometry', version='main'), # Maybe 'desispec' to mean computed rather than fetched from files?
                db.Version(package='redshift', version='daily/v0'),
                db.Version(package='tiles', version='trunk'),
                db.Version(package='specprod', version=specprod),
                db.Version(package='numpy', version=np.__version__),
                db.Version(package='astropy', version=astropy_version),
                db.Version(package='sqlalchemy', version=sqlalchemy_version),
                db.Version(package='desiutil', version=desiutil_version)]
    db.dbSession.add_all(versions)
    db.dbSession.commit()

## Read tiles file

In [4]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [9]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0) & (~tiles_table['PROGRAM'].mask))[0]
tiles_table[row_index]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80615,cmx,other,m33,cmxm33,4,3600.1,24.027,31.39,0.0,3658.5,3270.6,1000.0,obsend,3582.4,3658.5,3945.2,4469.3,other,0.9,20201216
80609,sv1,dark,lrgqso,cmxlrgqso,15,13500.7,150.12,2.206,0.0,8202.4,8317.8,4000.0,obsend,8103.8,8202.4,9070.2,12421.2,dark,0.9,20201217
80606,sv1,dark,elg,cmxelg,12,10800.7,36.448,-4.501,0.0,7179.2,7183.1,4000.0,obsend,6967.0,7179.2,7653.2,8686.3,dark,0.9,20201219
80620,sv1,dark,lrgqso,sv1lrgqso,13,11700.7,144.0,65.0,0.0,8228.4,7516.2,4000.0,obsend,7522.7,8228.4,7875.7,7957.3,dark,0.9,20201222
80611,sv1,bright,bgsmws,sv1bgsmws,13,3620.9,35.22,30.3,0.0,691.2,712.8,1000.0,obsstart,660.5,754.1,691.2,440.5,bright,0.9,20201222
80607,sv1,dark,lrgqso,cmxlrgqso,77,18804.9,0.0,0.0,0.0,10817.0,0.0,1000.0,obsend,9949.4,10817.0,10569.1,10520.4,dark,0.9,20201222
80622,sv1,dark,lrgqso,sv1lrgqso,8,7200.4,155.0,32.325,0.0,5543.6,5477.8,4000.0,obsend,5272.7,5543.6,5844.3,8490.2,dark,0.9,20201223
80617,sv1,bright,bgsmws,sv1bgsmws,22,6162.4,5.0,28.0,0.0,1451.9,1449.5,600.0,obsend,1356.3,1497.1,1451.9,1064.0,bright,0.9,20201223
80616,sv1,bright,bgsmws,sv1bgsmws,18,5106.2,356.0,29.0,0.0,1557.1,1575.5,600.0,obsend,1496.5,1694.1,1557.1,847.0,bright,0.9,20201223


In [6]:
tiles_table[row_index][:20]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80615,cmx,other,m33,cmxm33,4,3600.1,24.027,31.39,0.0,3658.5,3270.6,1000.0,obsend,3582.4,3658.5,3945.2,4469.3,other,0.9,20201216
80609,sv1,dark,lrgqso,cmxlrgqso,15,13500.7,150.12,2.206,0.0,8202.4,8317.8,4000.0,obsend,8103.8,8202.4,9070.2,12421.2,dark,0.9,20201217
80606,sv1,dark,elg,cmxelg,12,10800.7,36.448,-4.501,0.0,7179.2,7183.1,4000.0,obsend,6967.0,7179.2,7653.2,8686.3,dark,0.9,20201219
80620,sv1,dark,lrgqso,sv1lrgqso,13,11700.7,144.0,65.0,0.0,8228.4,7516.2,4000.0,obsend,7522.7,8228.4,7875.7,7957.3,dark,0.9,20201222
80611,sv1,bright,bgsmws,sv1bgsmws,13,3620.9,35.22,30.3,0.0,691.2,712.8,1000.0,obsstart,660.5,754.1,691.2,440.5,bright,0.9,20201222
80607,sv1,dark,lrgqso,cmxlrgqso,77,18804.9,0.0,0.0,0.0,10817.0,0.0,1000.0,obsend,9949.4,10817.0,10569.1,10520.4,dark,0.9,20201222
80622,sv1,dark,lrgqso,sv1lrgqso,8,7200.4,155.0,32.325,0.0,5543.6,5477.8,4000.0,obsend,5272.7,5543.6,5844.3,8490.2,dark,0.9,20201223
80617,sv1,bright,bgsmws,sv1bgsmws,22,6162.4,5.0,28.0,0.0,1451.9,1449.5,600.0,obsend,1356.3,1497.1,1451.9,1064.0,bright,0.9,20201223
80616,sv1,bright,bgsmws,sv1bgsmws,18,5106.2,356.0,29.0,0.0,1557.1,1575.5,600.0,obsend,1496.5,1694.1,1557.1,847.0,bright,0.9,20201223


In [10]:
load_tiles = db.Tile.convert(tiles_table, row_index=row_index)

In [11]:
load_tiles[:20]

[Tile(tileid=80615),
 Tile(tileid=80609),
 Tile(tileid=80606),
 Tile(tileid=80620),
 Tile(tileid=80611),
 Tile(tileid=80607),
 Tile(tileid=80622),
 Tile(tileid=80617),
 Tile(tileid=80616),
 Tile(tileid=80608),
 Tile(tileid=80612),
 Tile(tileid=80623),
 Tile(tileid=80638),
 Tile(tileid=80629),
 Tile(tileid=80635),
 Tile(tileid=80633),
 Tile(tileid=80640),
 Tile(tileid=80642),
 Tile(tileid=80641),
 Tile(tileid=80676)]

In [12]:
try:
    db.dbSession.add_all(load_tiles[:20])
    db.dbSession.commit()
except IntegrityError:
    print("ERROR: Some tiles are already loaded!")
    db.dbSession.rollback()

## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these.

In [13]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

NameError: name 'new_tile' is not defined

In [21]:
load_exposures = list()
for new_tile in load_tiles[:20]:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    assert len(row_index) > 0
    print(exposures_table[['NIGHT', 'MJD', 'EFFTIME_SPEC']][row_index])
    load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_exposures

 NIGHT        MJD          EFFTIME_SPEC   
-------- -------------- ------------------
20201216 59200.06640136   716.840956115723
20201216 59200.07964421  840.8275482177737
20201216 59200.11058356 1060.2101547241216
20201216 59200.12381137 1040.6450103759769
 NIGHT        MJD          EFFTIME_SPEC   
-------- -------------- ------------------
20201214 59198.43399897  301.4567306518556
20201214 59198.44720794  250.1291877746583
20201214 59198.46041661 297.44109649658213
20201214 59198.47356309  292.6512313842775
20201215 59199.50455879 398.24849319458025
20201215 59199.51829467   376.566352081299
20201215 59199.53212847  580.4486907958986
20201216 59200.46463512  672.6759353637698
20201216 59200.47864221  617.7882095336917
20201216 59200.49250742  908.0261367797855
20201216  59200.5063548  793.4541275024417
20201216 59200.52021614   836.430572509766
20201216 59200.53411147  778.9840789794924
20201217 59201.50105032   709.863888549805
20201217 59201.51562084 388.25950317382825
 NIGHT     

In [16]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [17]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

Should filter on TARGETID already loaded before creating the `potential_cat`.

In [ ]:
fiberassign_file = findfile('fiberassignsvn', tile=new_tile.tileid, readonly=True)
potential_targets_table = Table.read(fiberassign_file, format='fits', hdu='TARGETS')
no_sky_rows = no_sky(potential_targets_table)
potential_targets_table = Table(potential_targets_table[no_sky_rows])
potential_targets_table

In [ ]:
potential_tractorphot_already_loaded = db.dbSession.query(db.Photometry.targetid).filter(db.Photometry.targetid.in_(potential_targets_table['TARGETID'].tolist())).all()
potential_tractorphot_not_already_loaded = np.ones((len(potential_targets_table),), dtype=bool)
if len(potential_tractorphot_already_loaded) > 0:
    print("DEBUG: removing {0:d} objects already loaded.".format(len(potential_tractorphot_already_loaded)))
    for row in potential_tractorphot_already_loaded:
        potential_tractorphot_not_already_loaded[potential_targets_table['TARGETID'] == row[0]] = False

In [ ]:
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets_table['TARGETID'][potential_tractorphot_not_already_loaded]
potential_cat['TILEID'] = new_tile.tileid
potential_cat['TARGET_RA'] = potential_targets_table['RA'][potential_tractorphot_not_already_loaded]
potential_cat['TARGET_DEC'] = potential_targets_table['DEC'][potential_tractorphot_not_already_loaded]
# potential_cat['PETAL_LOC'] = potential_targets_table['PETAL_LOC'][potential_tractorphot_not_already_loaded]
potential_cat['SURVEY'] = new_tile.survey
potential_cat['PROGRAM'] = new_tile.program
# potential_cat

In [ ]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
# potential_targetphot

In [ ]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [ ]:
assert (np.where(potential_tractorphot['RELEASE'] == 0)[0] == np.where(potential_tractorphot['BRICKNAME'] == '')[0]).all()

In [ ]:
row_index = np.where(potential_tractorphot['BRICKNAME'] != '')[0]
load_photometry = db.Photometry.convert(potential_tractorphot, row_index=row_index)
# load_photometry[:20]

In [ ]:
if len(load_photometry) > 0:
    # db.dbSession.rollback()
    db.dbSession.add_all(load_photometry)
    db.dbSession.commit()

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [ ]:
load_rows = np.zeros((len(potential_targetphot),), dtype=bool)
if len(load_photometry) > 0:
    loaded_targetid = Table()
    loaded_targetid['TARGETID'] = np.array([r.targetid for r in load_photometry])
    loaded_targetid['LS_ID'] = np.array([r.ls_id for r in load_photometry])
    j = join(potential_targetphot['TARGETID', 'RELEASE'], loaded_targetid, join_type='left', keys='TARGETID')
    try:
        load_targetids = j['TARGETID'][j['LS_ID'].mask]
    except AttributeError:
        #
        # This means *every* TARGETID is already loaded.
        #
        pass
    else:
        unique_targetid, targetid_index = np.unique(potential_targetphot['TARGETID'].data, return_index=True)
        for t in load_targetids:
            load_rows[targetid_index[unique_targetid == t]] = True
# load_rows

In [ ]:
potential_targetphot_already_loaded = db.dbSession.query(db.Photometry.targetid).filter(db.Photometry.targetid.in_(potential_targetphot[load_rows]['TARGETID'].tolist())).all()
potential_targetphot_not_already_loaded = np.ones((len(potential_targetphot),), dtype=bool)
for row in potential_targetphot_already_loaded:
    potential_targetphot_not_already_loaded[potential_targetphot['TARGETID'] == row[0]] = False

In [ ]:
row_index = np.where(load_rows & potential_targetphot_not_already_loaded)[0]
load_targetphot = db.Photometry.convert(potential_targetphot, row_index=row_index)
# load_targetphot[:20]

In [ ]:
if len(load_targetphot) > 0:
    # db.dbSession.rollback()
    db.dbSession.add_all(load_targetphot)
    db.dbSession.commit()

In [ ]:
load_target = db.Target.convert(potential_targetphot, new_tile.survey, new_tile.tileid)
# load_target[:20]

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_target)
db.dbSession.commit()

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
redrock_files = list()
for spectrograph in range(10):
    redrock_file, redrock_exists = findfile('redrock_tile', groupname='cumulative', tile=new_tile.tileid, spectrograph=spectrograph, night=new_tile.lastnight, readonly=True, return_exists=True)
    if redrock_exists:
        redrock_files.append(redrock_file)

In [ ]:
load_ztile = list()
for rr in redrock_files:
    redrock_table, expfibermap = read_redrock(rr, group='cumulative', recoadd_fibermap=True, pertile=True)
    assert (expfibermap['TILEID'] == new_tile.tileid).all()
    firstnight = np.unique(expfibermap['NIGHT']).tolist()
    assert len(firstnight) == 1
    row_index = no_sky(redrock_table)
    load_ztile += db.Ztile.convert(redrock_table, new_tile.survey, new_tile.program, new_tile.tileid, firstnight[0], row_index=row_index)

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_ztile)
db.dbSession.commit()

## Load fiberassign and potential

In [ ]:
fiberassign_table = Table.read(fiberassign_file, format='fits', hdu='FIBERASSIGN')
potential_table = Table.read(fiberassign_file, format='fits', hdu='POTENTIAL_ASSIGNMENTS')

In [ ]:
row_index = no_sky(fiberassign_table)
load_fiberassign = db.Fiberassign.convert(fiberassign_table, new_tile.tileid, row_index=row_index)
# load_fiberassign

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_fiberassign)
db.dbSession.commit()

In [ ]:
row_index = no_sky(potential_table)
load_potential = db.Potential.convert(potential_table, new_tile.tileid, row_index=row_index)
# load_potential

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_potential)
db.dbSession.commit()

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
zall_tilecumulative = db.dbSession.query(db.Ztile).all()
zall_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in zall_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
nspec, primary = find_primary_spectra(zall_table)
zcat_nspec, zcat_primary = nspec.tolist(), primary.tolist()
for k, z in enumerate(zall_tilecumulative):
    z.zcat_nspec = zcat_nspec[k]
    z.zcat_primary = zcat_primary[k]
db.dbSession.commit()

In [ ]:
sv_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('sv1', 'sv2', 'sv3'))).all()
if len(sv_tilecumulative) > 0:
    sv_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in sv_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(sv_table)
    sv_nspec, sv_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(sv_tilecumulative):
        z.sv_nspec = sv_nspec[k]
        z.sv_primary = sv_primary[k]
    db.dbSession.commit()

In [ ]:
main_tilecumulative = db.dbSession.query(db.Ztile).filter(db.Ztile.survey.in_(('main', ))).all()
if len(main_tilecumulative) > 0:
    main_table = Table(list(zip(*[(z.targetid, z.zwarn, z.tsnr2_lrg) for z in main_tilecumulative])), names=('TARGETID', 'ZWARN', 'TSNR2_LRG'))
    nspec, primary = find_primary_spectra(main_table)
    main_nspec, main_primary = nspec.tolist(), primary.tolist()
    for k, z in enumerate(main_tilecumulative):
        z.main_nspec = main_nspec[k]
        z.main_primary = main_primary[k]
    db.dbSession.commit()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
q3c_updates = {'tile': 'tilera', 'exposure': 'tilera', 'photometry': 'ra', 'fiberassign': 'target_ra'}
for table in q3c_updates:
    db.q3c_index(table, ra=q3c_updates[table])